In [1]:
import sqlite3

In [2]:
# make queries

# Create a SQL connection to our SQLite database
con = sqlite3.connect("/home/angelo/Documents/Uni/Courses/Data Managment & Ethics/Integrated Assignment/database/assignment_db_2.sqlite")

cur = con.cursor()

# # The result of a "cursor.execute" can be iterated over by row
# for row in cur.execute('SELECT * FROM species;'):
#     print(row)

# # Be sure to close the connection
# con.close()






In [3]:
# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('''SELECT Year, PrimaryCategory, CrimeCountCurrentYear, ROUND(CAST((CrimeCountCurrentYear - LAG(CrimeCountCurrentYear) OVER (PARTITION BY PrimaryCategory)) AS REAL) / CAST(LAG(CrimeCountCurrentYear) OVER (PARTITION BY PrimaryCategory) AS REAL) * 100, 4) AS PCTChange
FROM
(SELECT DISTINCT vct.Year, ct.PrimaryCategory, COUNT (*) OVER( PARTITION BY PrimaryCategory, Year ORDER BY PrimaryCategory) AS CrimeCountCurrentYear 
FROM view_convenient_time as vct JOIN CrimeType AS ct
ON vct.IUCR == ct.IUCR);'''):
    print(row)

OperationalError: no such table: view_convenient_time

In [20]:
for row in cur.execute('SELECT * FROM CrimeType;'):
    print(row)

OperationalError: no such table: CrimeType

In [16]:
# Return all results of query
cur.execute('SELECT plot_id FROM plots WHERE plot_type="Control"')
cur.fetchall()

OperationalError: no such table: plots

In [6]:
cur.execute('SELECT * FROM CrimeType')

OperationalError: no such table: CrimeType